# Trabalho em Grupo I

```
Disciplina: Inteligência artificial 6º semestre
IFSP campus Campinas
Alunos: 
- Paulo Henrique Belucci - CP3001369
- Silvio da Col de Brito - CP300130X
Prof.: Dr. Samuel Martins
```

## Análise exploratória - Pandemia do Corona vírus no Brasil

---

### Descrição e motivação

A pandemia do Corona vírus teve início ao final do ano 2019 e, em pouco mais de um ano, é responsável por mais de 1,4 milhão de mortes em todo o mundo. Seus sintomas são parecidos com os sintomas da gripe, com destaque para a síndrome respiratória. Embora sua taxa de letalidade não seja tão alta, seu sintomas somados a um grande número de infectados tem se tornado um gravíssimo problema de saúde pública em todos os países, devido a sobrecarga de leitos de Unidade de Tratamento Intensivo (UTI) por aqueles casos se se agravam.

Muitos estudos e pesquisas tem sido desenvolvidos em prol de mapear a genética do vírus, entender as formas de contágio, testar tratamentos para pacientes infectados e, destaca-se, desenvolver uma vacina para imunizar a população interrompendo assim o ciclo de contágio.

Esta análise exploratória visa entender alguns aspectos da pandemia no Brasil, tais informações servem de insumo para elaboração de estratégias eficazes no combate ao avança da doença.



### Descrição da base de dados

O conjunto de dados foi extraído do [painel oficial](https://covid.saude.gov.br/) de COVID-19 no Brasil e cobre informações de todos os estados e municípios.

Cada registro contabiliza os casos novos, acumulados, recuperados por estado e município. A periodicidade é diária, ou seja, cada linha representa um dia.

As colunas presentes no _dataset_ são:
- regiao
- estado
- municipio
- coduf
- codmun
- codRegiaoSaude
- nomeRegiaoSaude
- data
- semanaEpi
- populacaoTCU2019
- casosAcumulado
- casosNovos
- obitosAcumulado
- obitosNovos
- Recuperadosnovos
- emAcompanhamentoNovos
- interior/metropolitana

Na sequência é realizada uma análise prévia sobre o dataset, seus tipos de dados, numero de registros, dados faltantes:

In [2]:
import pandas as pd

In [3]:
covid = pd.read_csv('HIST_PAINEL_COVIDBR_02jan2021.csv', delimiter=';')
covid.head()

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
0,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-25,9,210147125.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
1,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-26,9,210147125.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN
2,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-27,9,210147125.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN
3,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-28,9,210147125.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN
4,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-29,9,210147125.0,2.0,1.0,0.0,0.0,NaN,NaN,NaN


In [6]:
covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18187 entries, 0 to 18186
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   regiao                  18187 non-null  object 
 1   estado                  17873 non-null  object 
 2   municipio               9140 non-null   object 
 3   coduf                   18187 non-null  int64  
 4   codmun                  9422 non-null   float64
 5   codRegiaoSaude          9140 non-null   float64
 6   nomeRegiaoSaude         9140 non-null   object 
 7   data                    18187 non-null  object 
 8   semanaEpi               18187 non-null  int64  
 9   populacaoTCU2019        17904 non-null  float64
 10  casosAcumulado          18186 non-null  float64
 11  casosNovos              18186 non-null  float64
 12  obitosAcumulado         18186 non-null  float64
 13  obitosNovos             18186 non-null  float64
 14  Recuperadosnovos        260 non-null  

### Preparação da base de dados

Infelizmente as colunas de estados e municiṕios possuem muitos dados faltantes. No caso dos estados, alguns podem ser recuperados com base no respectivo código 'coduf'. Já no caso de municípios, os códigos também possuem muitos campos faltantes.